In [ ]:
from my_functions import *

import numpy as np

import pandas as pd

import matplotlib.pyplot as plt

import glob

In [ ]:
w_central = central_wavelength()
nb_fwhm_Arr = nb_fwhm(range(60))
w_lya = 1215.67

In [ ]:
## Load my QSO catalog

filename = '/home/alberto/almacen/Source_cats/QSO_100000_v6/data.csv'
data_qso = pd.read_csv(filename)

qso_flx = data_qso.to_numpy()[:, 1 : 60 + 1].T
qso_err = data_qso.to_numpy()[:, 60 + 1 : 120 + 1].T

EW_qso = data_qso['EW0'].to_numpy()
qso_zspec = data_qso['z'].to_numpy()

In [ ]:
## Load SF catalog

filename = '/home/alberto/almacen/Source_cats/LAE_10deg_z2-4_0/'
files = glob.glob(filename +'data*')
files.sort()
fi = []

for name in files:
    fi.append(pd.read_csv(name))

data = pd.concat(fi, axis=0, ignore_index=True)

sf_flx = data.to_numpy()[:, 1 : 60 + 1].T
sf_err = data.to_numpy()[:, 60 + 1 : 120 + 1].T + 1e-20

EW_sf = data['EW0'].to_numpy()
sf_zspec = data['z'].to_numpy()

In [ ]:
pm_flx = np.hstack((qso_flx, sf_flx))
pm_err = np.hstack((qso_err, sf_err))
zspec = np.concatenate((qso_zspec, sf_zspec))
EW_lya = np.concatenate((EW_qso, EW_sf))

N_sf = sf_flx.shape[1]
N_qso = qso_flx.shape[1]

qso_dL = cosmo.luminosity_distance(qso_zspec).to(u.cm).value
sf_dL = cosmo.luminosity_distance(sf_zspec).to(u.cm).value

sf_L = data['L_lya'].to_numpy()
qso_L = data_qso['L_lya'].to_numpy()

sf_flambda = 10 ** sf_L / (4*np.pi * sf_dL **2)
sf_flambda_err = np.zeros(N_sf)
qso_flambda = data_qso['F_line']
qso_flambda_err = np.abs(data_qso['F_line_err'])

L_lya = np.concatenate((qso_L, sf_L))
fline = np.concatenate((qso_flambda, sf_flambda))
fline_err = np.concatenate((qso_flambda_err, sf_flambda_err))

is_qso = np.concatenate((np.ones(N_qso), np.zeros(N_sf))).astype(bool)

N_sources = pm_flx.shape[1]

%xdel sf_flx
%xdel sf_err
%xdel qso_flx
%xdel qso_err
%xdel sf_zspec
%xdel qso_zspec
%xdel EW_sf
%xdel EW_qso
%xdel qso_dL
%xdel sf_L
%xdel qso_L
%xdel sf_flambda
%xdel qso_flambda
%xdel sf_flambda_err
%xdel qso_flambda_err
%xdel mock
%xdel data
%xdel data_qso

In [ ]:
# Lya search
cont_est_lya, cont_err_lya = estimate_continuum(
    pm_flx, pm_err, IGM_T_correct=True, N_nb_min=3, N_nb_max=25
)
line = is_there_line(pm_flx, pm_err, cont_est_lya, cont_err_lya, 20)
lya_lines, lya_cont_lines, line_widths = identify_lines(
    line, pm_flx, pm_err, first=True, return_line_width=True
)
lya_lines = np.array(lya_lines)

# Other lines
cont_est_other, cont_err_other = estimate_continuum(pm_flx, pm_err, IGM_T_correct=False)
line_other = is_there_line(pm_flx, pm_err, cont_est_other, cont_err_other,
    400, obs=True)
other_lines = identify_lines(line_other, pm_flx, pm_err)

# Compute z
z_Arr = np.zeros(N_sources)
z_Arr[np.where(np.array(lya_lines) != -1)] =\
    z_NB(np.array(lya_cont_lines)[np.where(np.array(lya_lines) != -1)])

nice_z = np.abs(z_Arr - zspec) < 0.12

%xdel cont_est_other
%xdel cont_err_other

In [ ]:
mag = flux_to_mag(pm_flx[-2], w_central[-2])
mag[np.isnan(mag)] = 99.

In [ ]:
mag_min = 16
mag_max = 24

nb_min = 3
nb_max = 15

nbs_to_consider = np.arange(nb_min, nb_max + 1)

nb_cut = (np.array(lya_lines) >= nb_min) & (np.array(lya_lines) <= nb_max)

z_min = (w_central[nb_min] - nb_fwhm_Arr[nb_min] * 0.5) / w_lya - 1
z_max = (w_central[nb_max] + nb_fwhm_Arr[nb_max] * 0.5) / w_lya - 1

z_cut = (z_min < z_Arr) & (z_Arr < z_max)
zspec_cut = (z_min < zspec) & (zspec < z_max)
ew_cut = EW_lya > 20
mag_cut = (mag > mag_min) & (mag < mag_max)

nice_lya = nice_lya_select(
    lya_lines, other_lines, pm_flx, pm_err, cont_est_lya, z_Arr
)
nice_lya = (nice_lya & z_cut & mag_cut)

In [ ]:
# z_Arr = zspec

# NB_z_Arr = z_NB(np.arange(56))
# lya_lines = np.ones(N_sources).astype(int) * -1
# for src in range(N_sources):
#     if (z_Arr[src] < 2) | (z_Arr[src] > 6):
#         continue
#     lya_lines[src] = np.argmin(np.abs(z_Arr[src] - NB_z_Arr))

In [ ]:
def plot_contours(maskkk, title='', nb_c=-3):
    fig, ax = plt.subplots(figsize=(7, 6))

    mask = maskkk[is_qso]
    Z, x, y = np.histogram2d(
        L_lya[is_qso][mask], L_Arr[is_qso][mask],
        bins=(np.linspace(42, 47, 30), np.linspace(42, 47, 30))
    )

    H_min = np.amin(Z)
    H_max = np.amax(Z)

    y_centers = 0.5 * (y[1:] + y[:-1])
    x_centers = 0.5 * (x[1:] + x[:-1])

    N_bins = 10000

    H_Arr = np.linspace(H_min, H_max, N_bins)[::-1]

    fact_up_Arr = np.zeros(N_bins)

    TOTAL_H = np.sum(Z)

    for iii in range(0, N_bins):

        mask = Z > H_Arr[iii]

        fact_up_Arr[iii] = np.sum(Z[mask]) / TOTAL_H

    H_value_68 = np.interp(0.683, fact_up_Arr, H_Arr) # 1 sigma
    H_value_95 = np.interp(0.954, fact_up_Arr, H_Arr) # 2 sigma
    H_value_99 = np.interp(0.997, fact_up_Arr, H_Arr) # 3 sigma

    ax.contour(
        x_centers, y_centers, Z.T, levels=[H_value_99, H_value_95, H_value_68],
        colors='C0'
    )

    mask = ~is_qso & maskkk
    Z, x, y = np.histogram2d(
        L_lya[mask], L_Arr[mask],
        bins=(np.linspace(42, 47, 30), np.linspace(42, 47, 30))
    )

    H_min = np.amin(Z)
    H_max = np.amax(Z)

    y_centers = 0.5 * (y[1:] + y[:-1])
    x_centers = 0.5 * (x[1:] + x[:-1])

    N_bins = 10000

    H_Arr = np.linspace(H_min , H_max , N_bins )[::-1]

    fact_up_Arr = np.zeros(N_bins)

    TOTAL_H = np.sum(Z)

    for iii in range(0, N_bins):

        mask = Z > H_Arr[iii]

        fact_up_Arr[iii] = np.sum(Z[ mask ]) / TOTAL_H

    H_value_68 = np.interp(0.683, fact_up_Arr, H_Arr) # 1sigma
    H_value_95 = np.interp(0.954, fact_up_Arr, H_Arr) # 2sigma
    H_value_99 = np.interp(0.997, fact_up_Arr, H_Arr) # 2sigma

    ax.contour(
        x_centers, y_centers, Z.T, levels=[H_value_99, H_value_95, H_value_68],
        colors='C1'
    )

    x = np.linspace(40, 48, 100)
    ax.plot(x, x, linestyle='--', color='red', label='1:1')

    ax.set_ylabel('Retrieved $\log L$', fontsize=15)
    ax.set_xlabel('Real $\log L$', fontsize=15)

    ax.set_ylim((42, 47))
    ax.set_xlim((42, 47))

    if len(title) > 0:
        ax.set_title(title, fontsize=20)

    # Detec lim

    detec_lim = np.vstack(
        (
            pd.read_csv('csv/5sigma_depths_NB.csv', header=None),
            pd.read_csv('csv/5sigma_depths_BB.csv', header=None)
        )
    )[:, 1]

    flambda_lim = mag_to_flux(detec_lim[nb_c], w_central[nb_c]) * 3

    ew0_lim = 20 # A
    z = w_central[nb_c] / 1215.67 - 1
    Fline_lim = ew0_lim * flambda_lim * (1 + z)
    dL = cosmo.luminosity_distance(z).to(u.cm).value
    L_lim = np.log10(Fline_lim * 4*np.pi * dL**2)

    ax.axhline(L_lim, ls='--', color='green', label='L limit')

    ax.legend(fontsize=15)
    # plt.savefig(f'/home/alberto/Desktop/{title}')
    plt.show()

In [ ]:
EW_nb_Arr, EW_nb_e, L_Arr, L_e_Arr, flambda, flambda_e = EW_L_NB(
    pm_flx, pm_err, cont_est_lya, cont_err_lya, z_Arr, lya_lines, N_nb=0
)

In [ ]:
tcurves = load_tcurves(load_filter_tags())
nb = 20
nb_name = tcurves['tag'][nb]
plot_contours((mag < 23) & (lya_lines == nb), f'{nb_name}, retrieved z', nb)

In [ ]:
def plot_L_hist(mask, title):
    fig, ax = plt.subplots(figsize=(7, 6))

    ax.hist(L_Arr[~is_qso & mask], bins=np.linspace(42, 46, 30), color='C1', alpha=0.4)
    ax.hist(L_lya[~is_qso & mask], bins=np.linspace(42, 46, 30), color='C1', histtype='step')

    ax.hist(L_Arr[is_qso & mask], bins=np.linspace(42, 46, 30), color='C0', alpha=0.4)
    ax.hist(L_lya[is_qso & mask], bins=np.linspace(42, 46, 30), color='C0', histtype='step')

    # plt.hist(L_Arr[mask], bins=np.linspace(42, 46, 30), color='k', histtype='step')

    ax.set_title(title, fontsize=20)
    ax.set_ylabel('N', fontsize=15)
    ax.set_xlabel('log L', fontsize=15)

    plt.show()

mag_hist_lims = np.arange(18.5, 24.5, 0.5)
for i in range(len(mag_hist_lims) - 1):
    mask = (
        (mag < mag_hist_lims[i + 1]) & (mag > mag_hist_lims[i])
        & nice_lya & nice_z
        & (lya_lines > 6) & (lya_lines < 20)
    )
    title = f'{mag_hist_lims[i]} < r < {mag_hist_lims[i + 1]}'
    plot_L_hist(mask, title)

print(f'QSOs: {count_true(mask & is_qso)}')
print(f'SFs: {count_true(mask & ~is_qso)}')

In [ ]:
from scipy.stats import median_abs_deviation as mad

def plot_err_hist(mask, title):
    fig, ax = plt.subplots(figsize=(7, 6))

    dist_qso = (10 ** L_Arr[mask & is_qso] - 10 ** L_lya[mask & is_qso])\
        / L_e_Arr[mask & is_qso]
    dist_sf = (10 ** L_Arr[mask & ~is_qso] - 10 ** L_lya[mask & ~is_qso])\
        / L_e_Arr[mask & ~is_qso]

    ax.hist(dist_qso, bins=np.linspace(-5, 15, 15))
    ax.hist(dist_sf, bins=np.linspace(-5, 15, 20))

    ax.set_xlabel('(L_ret - L_real) / $\sigma$', fontsize=15)

    plt.show()

    print(f'QSO std = {mad(dist_qso, nan_policy="omit"):0.2f}')
    print(f'SF std = {mad(dist_sf, nan_policy="omit"):0.2f}')

mask = nice_lya & nice_z & (mag < 24) & (lya_lines == 10)
plot_err_hist(mask, '')

In [ ]:
fig, ax = plt.subplots(figsize=(11, 6))

mask =  (mag < 23) & ~is_qso & (lya_lines > 7) & (lya_lines < 20)
ax.errorbar(EW_lya[mask], EW_nb_Arr[mask], fmt='s', color='k', yerr=EW_nb_e[mask], ecolor='gray')
ax.errorbar(
    EW_lya[mask & nice_lya & nice_z], EW_nb_Arr[mask & nice_lya & nice_z],
    fmt='s', color='red', yerr=EW_nb_e[mask & nice_lya & nice_z], ecolor='red', zorder=99
)


x = np.linspace(1, 1e4, 100)
ax.plot(x, x, linestyle='--', color='red', label='1:1')

ax.set_ylabel('Retrieved EW0', fontsize=15)
ax.set_xlabel('Real EW0', fontsize=15)

ax.set_yscale('log')
ax.set_xscale('log')
ax.set_title('SF, 1NB, r < 23', fontsize=20)

ax.set_ylim((1, 1e3))
ax.set_xlim((5, 2e2))

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(11, 6))

mask_fline_relerr = (fline_err / fline < 0.2)
mask = (mag < 23) & (lya_lines == 10) & is_qso & mask_fline_relerr

EW_lya_err = EW_lya / fline * fline_err

ax.errorbar(
    EW_lya[mask], EW_nb_Arr[mask],
    fmt='s', color='k', yerr=EW_nb_e[mask], ecolor='gray')
ax.errorbar(
    EW_lya[mask & nice_lya & nice_z], EW_nb_Arr[mask & nice_lya & nice_z],
    fmt='s', color='red', yerr=EW_nb_e[mask & nice_lya & nice_z], ecolor='red', zorder=99
)


x = np.linspace(1, 1e4, 100)
ax.plot(x, x, linestyle='--', color='red', label='1:1')

ax.set_ylabel('Retrieved EW0', fontsize=15)
ax.set_xlabel('Real EW0', fontsize=15)

ax.set_yscale('log')
ax.set_xscale('log')
ax.set_title('QSO, 1NB', fontsize=20)

ax.set_ylim((2, 1e3))
ax.set_xlim((1, 1e3))

plt.show()